In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("records").getOrCreate()


ConnectionRefusedError: [Errno 111] Connection refused

In [8]:
import time
from pyspark.sql.functions import col, when, rand, expr
from pyspark.sql import SparkSession

# Initialize Spark session with memory optimizations
spark = SparkSession.builder \
    .appName("Optimized Iceberg Update") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.memoryOverhead", "2g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

# Load the Iceberg table
df = spark.table("demo.nyc.taxis_10M_50COLUMNS")

# Debugging: Print total row count
total_rows = df.count()
print(f"Total rows in table: {total_rows}")

# Get update percentage from user
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100
num_rows = max(1, int(total_rows * update_percentage))  # Ensure at least 1 row is updated

# Define batch size
batch_size = 500  # Reduced batch size to optimize memory usage
num_batches = max(1, (num_rows // batch_size) + (1 if num_rows % batch_size else 0))  # Ensure at least 1 batch

print(f"Updating {num_rows} rows (~{update_percentage*100}%) in {num_batches} batches...")

st = time.time()

for batch_num in range(num_batches):
    print(f"Processing batch {batch_num + 1}/{num_batches}...")

    # Select a batch of unique row IDs without collecting to the driver
    sampled_df = df.select("extra_col_0").distinct().orderBy(rand()).limit(min(batch_size, num_rows))

    # Merge updates instead of overwriting partitions
    updated_batch = df.alias("source").join(sampled_df.alias("updates"), "extra_col_0", "inner") \
        .withColumn("extra_col_1", col("extra_col_1") + 10)

    # Optimize writing using append mode or merge
    updated_batch.repartition(100).writeTo("demo.nyc.taxis_10M_50COLUMNS").append()

    print(f"Batch {batch_num + 1}/{num_batches} updated.")

end = time.time() - st
print(f"\nTotal update time for {num_batches} batches: {end:.2f} sec")


Total rows in table: 10004500


Enter update percentage (e.g., 1 for 1%):  1


Updating 100045 rows (~1.0%) in 201 batches...
Processing batch 1/201...


25/03/04 09:17:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:17:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:17:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:17:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:17:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:17:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:17:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:17:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


Batch 1/201 updated.
Processing batch 2/201...


...

25/03/04 09:42:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:42:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:42:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:42:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:42:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:42:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:42:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:42:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


Batch 9/201 updated.
Processing batch 10/201...


[6058.891s][warning][gc,alloc] Executor task launch worker for task 231.0 in stage 305.0 (TID 10072): Retried waiting for GCLocker too often allocating 1048578 words


25/03/04 09:45:04 ERROR Executor: Exception in task 231.0 in stage 305.0 (TID 10072)
java.lang.OutOfMemoryError: Java heap space
25/03/04 09:45:04 WARN TaskSetManager: Lost task 231.0 in stage 305.0 (TID 10072) (5e0ed98abed9 executor driver): java.lang.OutOfMemoryError: Java heap space

25/03/04 09:45:04 ERROR TaskSetManager: Task 231 in stage 305.0 failed 1 times; aborting job
25/03/04 09:45:04 ERROR SparkUncaughtExceptionHandler: Uncaught exception in thread Thread[Executor task launch worker for task 231.0 in stage 305.0 (TID 10072),5,main]
java.lang.OutOfMemoryError: Java heap space
25/03/04 09:45:04 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-244b373c-a8e3-4302-95fa-49829b358285/15/temp_shuffle_ed1e3a84-2ee4-4891-badc-0569d2bb0869
25/03/04 09:45:04 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-244b373c-a8e3-4302-95fa-49829b358285/3f/temp_shuffle_6327d217-3c86-4b4b-9eeb-a525b91abbb9
25/03/04 09:45:04 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-2

Py4JJavaError: An error occurred while calling o497.append.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 231 in stage 305.0 failed 1 times, most recent failure: Lost task 231.0 in stage 305.0 (TID 10072) (5e0ed98abed9 executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAG

In [9]:
import time
from pyspark.sql.functions import expr, rand
from pyspark.sql import SparkSession

# Initialize Spark session with memory optimizations
spark = SparkSession.builder \
    .appName("Optimized Iceberg Update") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "12g") \
    .config("spark.executor.memoryOverhead", "4g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.memory.fraction", "0.7") \
    .config("spark.sql.shuffle.partitions", "100") \
    .config("spark.sql.autoBroadcastJoinThreshold", "50MB") \
    .getOrCreate()

# Load the Iceberg table
df = spark.table("demo.nyc.taxis_10M_50COLUMNS")

# Debugging: Print total row count
total_rows = df.count()
print(f"Total rows in table: {total_rows}")

# Get update percentage from user
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100
num_rows = max(1, int(total_rows * update_percentage))  # Ensure at least 1 row is updated

# Define batch size dynamically
batch_size = min(500, num_rows // 10)  # Adjust batch size based on total updates
num_batches = max(1, (num_rows // batch_size) + (1 if num_rows % batch_size else 0))  # Ensure at least 1 batch

print(f"Updating {num_rows} rows (~{update_percentage*100}%) in {num_batches} batches...")

st = time.time()

for batch_num in range(num_batches):
    print(f"Processing batch {batch_num + 1}/{num_batches}...")
    
    # Select a batch of unique row IDs without collecting to the driver
    sampled_df = df.select("extra_col_0").distinct().orderBy(rand()).limit(min(batch_size, num_rows))
    
    # Use MERGE INTO for efficient updates
    spark.sql(f'''
        MERGE INTO demo.nyc.taxis_10M_50COLUMNS AS source
        USING (SELECT extra_col_0 FROM sampled_df) AS updates
        ON source.extra_col_0 = updates.extra_col_0
        WHEN MATCHED THEN
        UPDATE SET source.extra_col_1 = source.extra_col_1 + 10
    ''')

    print(f"Batch {batch_num + 1}/{num_batches} updated.")

end = time.time() - st
print(f"\nTotal update time for {num_batches} batches: {end:.2f} sec")


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: SparkSession$ does not exist in the JVM

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
